In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp plugin.listeners

In [ ]:
# export

import time
import os
import signal
from threading import Thread

In [ ]:
# export
def status_listener(client, run_id, status, callback, interval=5, verbose=False):
    """
    Listens to status field of plugin, and executes callback when run.status == status.
    """
    if verbose:
        print(f"Listening for status='{status}' on Item {run_id}")
    while True:
        time.sleep(interval)
        try:
            run = client.get(run_id)
            if verbose:
                print("run status:", run.status)
            if run.status == status:
                callback()
        except Exception as e:
            print(f"Could not get run in status listener: {e}")

In [ ]:
# export
def abort_plugin_callback():
    print("Status aborted, killing plugin...")
    pid = os.getpid()
    os.kill(pid, signal.SIGABRT)

def get_abort_plugin_listener(client, run_id, **kwargs):
    listener = Thread(
        target=status_listener,
        args=(client, run_id, "aborted", abort_plugin_callback),
        kwargs=kwargs
    )
    listener.start()
    return listener

In [ ]:
# hide
# skip
from pymemri.pod.client import PodClient
from pymemri.data.schema import PluginRun

def test_callback():
    """Print Callback and kill listener."""
    print("Callback")
    os.kill(os.getpid(), signal.SIGABRT)

client = PodClient()
run = PluginRun(containerImage="", status="started")
client.create(run)

listener = get_abort_plugin_listener(client, run.id, verbose=True)

for i in range(10):
    time.sleep(1)
    if i == 5:
        run = client.get(run.id)
        run.status = "aborted"
        client.update_item(run)

Listening for status='aborted' on Item cCef7eB50cB6fBb6e8Fd636aCcCFFd6a
run status: started


In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted basic.ipynb.
Converted cvu.utils.ipynb.
Converted data.photo.ipynb.
Converted index.ipynb.
Converted itembase.ipynb.
Converted plugin.authenticators.credentials.ipynb.
Converted plugin.authenticators.oauth.ipynb.
Converted plugin.listeners.ipynb.
Converted plugin.pluginbase.ipynb.
Converted plugin.states.ipynb.
Converted plugins.authenticators.password.ipynb.
Converted pod.client.ipynb.
Converted pod.db.ipynb.
Converted pod.utils.ipynb.
Converted test_schema.ipynb.
Converted test_utils.ipynb.
